# 3 eme Version - Amélioration du cadre d'analyse

In [1]:
import numpy as np
import random 
import simpy
import time
import csv
from collections import namedtuple
from datetime import datetime
import pandas as pd

L'objectif est d'améliorer le cadre de simulation de la version précédente en y implémentant un niveau d'aléatoiriété supplémentaire. En effet il faut implémenter le fait que le nombre de panneaux pouvant proposer des bid requests est variable et évolue dynamiquement. Ainsi, le délai entre chaque bid request ne suit plus seulement une loi à paramètre constant mais à paramètre variable. L'intérêt est ainsi de simuler le fait qu'à certaines heures de la journée le délai entre deux br est beaucoup plus court que d'autres heures.
    
Le second objectif est de pouvoir simuler ce qu'il s'est passé sur un jour, puis une semaine puis un mois par exemple et stocker les résultats dans un fichier (csv ou json par exemple) afin de réutiliser ces résultats dans la mise en place d'un algorithme de pacing plus élaboré que précédemment.

### Etape 1 - Générer les fonctions des variables aléatoires ainsi que les constantes

In [2]:
# Constantes globales
nombre_jours = 1 
nombre_panneaux = 10000

In [3]:
def panneaux_dispo(nb_heures, total_panneaux):
    prop = np.random.normal(loc=0.8, scale=0.2, size=nb_heures)
    pann_dispo = list(map(lambda x: int(total_panneaux*x if x<=1 else total_panneaux), prop))
    return pann_dispo

La difficulté majeure pour cette simulation est de trouver une bonne approche afin de simuler le fait que le $\lambda$  de la loi de Poisson soit lui même une variable aléatoire (qui pourrait varier chaque heure par exemple) et donc de faire le lien entre cette variable aléatoire et le temps entre chaque bid request (plus il y a de panneaux dispo sur une heure donnée et plus le délai entre deux br est court)

In [4]:
def lambda_br(pann_dispo):
    lam = list(map(lambda x: int((1000-x)/50) if x<950 else 1, pann_dispo))
    return lam

In [5]:
def delai(lam):
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (2% de chance d'en avoir un)
    #if not random.random() < 0.98:
     #   secondes = np.random.poisson(lam + 1000)
    return secondes

In [6]:
def imps():
    lam = int(np.random.normal(loc=4, scale=2, size=1))
    if lam < 1:
        lam = 1
    nb_imp = np.random.poisson(lam)
    return nb_imp

### Etape 2 - Générer la nouvelle simulation

In [7]:
def open_rtb(env, P, timestampnow):
    dispo = panneaux_dispo(10, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').minute
    i = 0
    while True:
        rt = (timestampnow+600)-env.now
        print(f"Remaining {rt} seconds")
        #Timestamp de la br
        time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
        print(f"Bid request at {time}")
        
        # Nombre d'impressions
        nb_imp = imps()
        print(f"{nb_imp} impressions")
        Prix = P * nb_imp
        print(f"Price of {Prix}€")
        
        # Detecter quelle heure est-t-il
        if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').minute != current_hour:
            i += 1 
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').minute
        
        # Temps avant la prochaine BR
        # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
        try:
            print(f"Lambda égal a {lam_secondes[i]}\n")
            time_before_next = delai(lam_secondes[i])
        except IndexError:
            print(f"Lambda égal a {lam_secondes[i-1]}\n")
            time_before_next = delai(lam_secondes[i-1])
            
        # Fin de la simulation
        if rt < time_before_next:
            print("\n Fin de la simulation !")
            
        
        yield env.timeout(time_before_next)

In [60]:
timestampnow = int(time.time())
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, 1, timestampnow))

In [62]:
env.run(until=timestampnow + 600)

ValueError: until(=1590505977) must be > the current simulation time.

In [8]:
def delai(lam):
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (1% de chance d'en avoir un)
    if not random.random() < 0.99:
        secondes = np.random.poisson(lam + 1000)
    return secondes

In [9]:
def open_rtb(env, P, timestampnow, nb_heures):
    dispo = panneaux_dispo(nb_heures, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    i = 0
    identifiant = 0
    while True:
        identifiant += 1
        Identifiants.append(identifiant)
        rt = (timestampnow+nb_heures*3600)-env.now
        
        #Timestamp de la br
        time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
        Timestampstring.append(time)
        Timestamps.append(env.now)
        
        # Nombre d'impressions
        nb_imp = imps()
        Impressions.append(nb_imp)
        Prix = P * nb_imp
        Prices.append(Prix)
        
        # Detecter quelle heure est-t-il
        if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
            i += 1 
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
        
        # Temps avant la prochaine BR
        # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
        try:
            time_before_next = delai(lam_secondes[i])
        except IndexError:
            time_before_next = delai(lam_secondes[i-1])
            
        # Fin de la simulation
        if rt < time_before_next:
            print(f"Fin de la simulation à {time} !")
        
        yield env.timeout(time_before_next)

In [10]:
timestampnow = int(time.time())
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, 1, timestampnow, 5))

In [11]:
Identifiants = []
Timestampstring = []
Timestamps = []
Impressions = []
Prices = []
env.run(until=timestampnow + 5*3600)

Fin de la simulation à 05-28-2020 15:39:48 !


In [12]:
len(Identifiants)

1806

### Etape 3 - Stocker les résultats 

On va tenter premièrement de stocker les résultats au format csv (format qui pourrait être suffisant étant donné le grand nombre de lignes mais le très faible nombre de colonnes)

In [13]:
def open_rtb(env, P, timestampnow, nb_heures, bidrequests, data):
    dispo = panneaux_dispo(nb_heures, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    i = 0
    identifiant = 0
    while True:
        identifiant += 1
        rt = (timestampnow+nb_heures*3600)-env.now
        
        #Timestamp de la br
        time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
        
        # Nombre d'impressions
        nb_imp = imps()
        Prix = P * nb_imp
        
         # Stocker les résultats dans le namedtuple puis ajout à une liste
        resultats = bidrequests(
            identifiant = identifiant,
            timestamp = env.now,
            timestamp_string = time,
            nombre_impressions = nb_imp,
            prix_impression = P,
            prix_total = Prix
        )
        data.append(resultats)
        
        # Detecter quelle heure est-t-il
        if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
            i += 1 
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
        
        # Temps avant la prochaine BR
        # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
        try:
            time_before_next = delai(lam_secondes[i])
        except IndexError:
            time_before_next = delai(lam_secondes[i-1])
       
        
        # Fin de la simulation
        if rt < time_before_next:
            print(f"Fin de la simulation à {time} !")
        
        yield env.timeout(time_before_next)

In [14]:
def sauvegarde(liste, nom_de_fichier):
    with open(nom_de_fichier, "w", encoding="utf8") as fichier:
        #Recuperer le nom des colonnes pour la première ligne
        premier, *_ = liste
        ecrivain = csv.DictWriter(fichier, premier._fields)
        ecrivain.writeheader()
        for br in liste:
            # On ecrit chaque ligne comme un ordered dict
            ecrivain.writerow(br._asdict())

In [15]:
timestampnow = int(time.time())
# Générer un named tuple pour stocker les résultats
bidrequests =  namedtuple(
    "bidrequests", 
    (
        "identifiant", 
        "timestamp", 
        "timestamp_string",
        "nombre_impressions",
        "prix_impression",
        "prix_total"
    )
                    )
# Liste pour stocker les résultats du namedtuple
data = list()
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, 1, timestampnow, 5, bidrequests, data))

In [16]:
env.run(until=timestampnow + 5*3600)

Fin de la simulation à 05-28-2020 15:40:25 !


In [17]:
sauvegarde(data, 'test.csv')

In [18]:
df = pd.read_csv('test.csv', index_col = 0)

In [19]:
df.head()

,timestamp,timestamp_string,nombre_impressions,prix_impression,prix_total
identifiant,,,,,
1,1590655231,05-28-2020 10:40:31,2,1,2
2,1590655231,05-28-2020 10:40:31,4,1,4
3,1590655232,05-28-2020 10:40:32,3,1,3
4,1590655233,05-28-2020 10:40:33,2,1,2
5,1590655234,05-28-2020 10:40:34,2,1,2


### Etape 4 - Simulations sur plusieurs jours

L'idée est de créer un environnement de simulation qui soit vraiment calqué sur un fonctionnement en temps réel. C'est-à-dire que si l'on suppose que les br peuvent être reçues entre 6h et 20h, alors la simulation "tourne" classiquement pendant ces horaires là et en dehors alors elle tournerait "dans le vide" mais le temps lui s'écoulerait quand même. Il y aurait alors quelque chose comme 3 états de la simulation : 

1. Juste avant l'ouverture, setup du cadre (affluence par heure des br etc.) 
2. Simulation de réception des br pendant la journée en fonction du cadre du jour
3. Etat "silencieux" ou rien ne se passe jusqu'à la prochaine journée

In [62]:
def total_sec(nb_jours):
    tot = nb_jours * 86400
    return tot

In [61]:
def open_rtb(env, P, timestampnow, nb_jours, bidrequests, data):
    dispo = panneaux_dispo(14, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    i = 0
    identifiant = 0
    while True:
        # Déterminer dans quel état on est actuellement
        if current_hour >= 6 and current_hour < 20:
            if setup:
                setup = False
                dispo = panneaux_dispo(14, 1000)
                lam_secondes = lambda_br(dispo)
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
                i = 0
            
            # Génération d'une br
            identifiant += 1
            rt = (timestampnow+total_sec(nb_jours))-env.now

            #Timestamp de la br
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")

            # Nombre d'impressions
            nb_imp = imps()
            Prix = P * nb_imp

             # Stocker les résultats dans le namedtuple puis ajout à une liste
            resultats = bidrequests(
                identifiant = identifiant,
                timestamp = env.now,
                timestamp_string = time,
                nombre_impressions = nb_imp,
                prix_impression = P,
                prix_total = Prix
            )
            data.append(resultats)

            # Detecter quelle heure est-t-il
            if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
                i += 1 
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour

            # Temps avant la prochaine BR
            # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
            try:
                time_before_next = delai(lam_secondes[i])
            except IndexError:
                time_before_next = delai(lam_secondes[i-1])


            # Fin de la simulation
            if rt < time_before_next:
                print(f"Fin de la simulation à {time} !")

            yield env.timeout(time_before_next)
            
        else:
            setup = True
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
            rt = (timestampnow+total_sec(nb_jours))-env.now
            if rt <= 1:
                print(f"Fin de la simulation à {time} !")
            yield env.timeout(1)  

In [64]:
timestampnow = int(time.time()-3600*6)
# Générer un named tuple pour stocker les résultats
bidrequests =  namedtuple(
    "bidrequests", 
    (
        "identifiant", 
        "timestamp", 
        "timestamp_string",
        "nombre_impressions",
        "prix_impression",
        "prix_total"
    )
                    )
# Liste pour stocker les résultats du namedtuple
data = list()
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, 1, timestampnow, 5, bidrequests, data))

In [65]:
env.run(until=timestampnow + total_sec(5))

Fin de la simulation à 06-02-2020 05:40:15 !


In [66]:
len(data)

18945

In [67]:
sauvegarde(data, 'donnees.csv')